In [1]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister 
from qiskit.visualization import plot_histogram
from matplotlib import pyplot as plt
import numpy as np
import math
import array
from fractions import Fraction

from QuantumRingsLib import QuantumRingsProvider
from quantumrings.toolkit.qiskit import QrBackendV2

In [2]:
# load API keys
from dotenv import load_dotenv
import os

load_dotenv()
QR_TOKEN = os.getenv("QR_TOKEN")
QR_NAME = os.getenv("QR_NAME")

# Acquire the Quantum Rings Provider
qr_provider = QuantumRingsProvider(token=QR_TOKEN, name=QR_NAME)

In [3]:
class ShorsFactorizer:
    def __init__(self, semiprime, provider : QuantumRingsProvider):
        self.semiprime = semiprime
        self.circuit_size = np.ceil(np.log2(semiprime)).astype(int)
        self.counts = None
        self.coprimes = list(filter(lambda x : np.gcd(x, self.semiprime) == 1, range(1, self.semiprime)))
        np.random.shuffle(self.coprimes)
        self.counts = dict()
        self.provider = provider
        self.factors = []

    def _modinv(self, a, m):
        """Returns the modular inverse"""
        return pow(a, -1, m)

    def _get_angles(self, a, N):
        """Returns phase angles for TODO: fill this in"""
        s = bin(int(a))[2:].zfill(N)
        angles = np.zeros([N])
        for i in range(0, N):
            for j in range(i, N):
                if s[j] == "1":
                    angles[N - i - 1] += np.power(2.0, -(j - i))
            angles[N - i - 1] *= np.pi
        return angles

    def _ccphase(
        self,
        circuit: QuantumCircuit,
        angle: float,
        control_1: int,
        control_2: int,
        target_qubit: int,
    ):
        """Adds double controlled phase gate"""
        circuit.cp(angle / 2, control_1, target_qubit)
        circuit.cx(control_2, control_1)
        circuit.cp(-angle / 2, control_1, target_qubit)
        circuit.cx(control_2, control_1)
        circuit.cp(angle / 2, control_2, target_qubit)

    def _create_QFT(
        self,
        circuit: QuantumCircuit,
        source_reg: QuantumRegister,
        circuit_size: int,
        with_swaps: int,
        threshold=None,
    ):
        i = circuit_size - 1
        if threshold is None:
            #threshold = np.log2(circuit_size)
            threshold = 100
        """ Apply the H gates and Cphases"""
        """ The Cphases with |angle| < threshold are not created because they do
        nothing. The threshold is put as being 0 so all CPhases are created,
        but the clause is there so if wanted just need to change the 0 of the
        if-clause to the desired value """
        while i >= 0:
            circuit.h(source_reg[i])
            j = i - 1
            while j >= 0 and i - j <= threshold:
                if (np.pi) / (pow(2, (i - j))) > 0:
                    circuit.cp(
                        (np.pi) / (pow(2, (i - j))), source_reg[i], source_reg[j]
                    )
                    j = j - 1
            i = i - 1

        """ If specified, apply the Swaps at the end """
        if with_swaps == 1:
            i = 0
            while i < ((circuit_size - 1) / 2):
                circuit.swap(source_reg[i], source_reg[circuit_size - 1 - i])
                i = i + 1

    def _create_inverse_QFT(
        self,
        circuit: QuantumCircuit,
        source_reg,
        circuit_size: int,
        with_swaps: int,
        threshold=None,
    ):
        if threshold is None:
            #threshold = np.log2(circuit_size)
            threshold = 100
        """If specified, apply the Swaps at the beggining"""
        if with_swaps == 1:
            i = 0
            while i < ((circuit_size - 1) / 2):
                circuit.swap(source_reg[i], source_reg[circuit_size - 1 - i])
                i = i + 1

        """ Apply the H gates and Cphases"""
        """ The Cphases with |angle| < threshold are not created because they do
        nothing. The threshold is put as being 0 so all CPhases are created,
        but the clause is there so if wanted just need to change the 0 of the
        if-clause to the desired value """
        i = 0
        while i < circuit_size:
            circuit.h(source_reg[i])
            if i != circuit_size - 1:
                j = i + 1
                y = i
                while y >= 0 and j - y <= threshold:
                    if (np.pi) / (pow(2, (j - y))) > 0:
                        circuit.cp(
                            -(np.pi) / (pow(2, (j - y))), source_reg[j], source_reg[y]
                        )
                        y = y - 1
            i = i + 1

    def _phiADD(self, circuit, target_reg, addend, semiprime, inv):
        """Fourier space addition"""
        angle = self._get_angles(addend, semiprime)
        for i in range(0, semiprime):
            if inv == 0:
                circuit.p(angle[i], target_reg[i])
            else:
                circuit.p(-angle[i], target_reg[i])

    def _cphiADD(self, circuit, target_reg, control, addend, semiprime, inv):
        """Single controlled Fourier space addition"""
        angle = self._get_angles(addend, semiprime)
        for i in range(0, semiprime):
            if inv == 0:
                circuit.cp(angle[i], control, target_reg[i])
            else:
                circuit.cp(-angle[i], control, target_reg[i])

    def _ccphiADD(self, circuit, target_reg, control_1, control_2, addend, circuit_size, inv):
        """Doubly controlled Fourier space addition"""
        angle = self._get_angles(addend, circuit_size)
        for i in range(0, circuit_size):
            if inv == 0:
                self._ccphase(circuit, angle[i], control_1, control_2, target_reg[i])
            else:
                self._ccphase(circuit, -angle[i], control_1, control_2, target_reg[i])

    def _ccphiADDmodN(self, circuit, target_reg, control_1, control_2, aux_reg, addend, semiprime, circuit_size):
        """Doubly controlled modular addition by a"""
        self._ccphiADD(circuit, target_reg, control_1, control_2, addend, circuit_size, 0)
        self._phiADD(circuit, target_reg, semiprime, circuit_size, 1)
        self._create_inverse_QFT(circuit, target_reg, circuit_size, 0)
        circuit.cx(target_reg[circuit_size - 1], aux_reg)
        self._create_QFT(circuit, target_reg, circuit_size, 0)
        self._cphiADD(circuit, target_reg, aux_reg, semiprime, circuit_size, 0)

        self._ccphiADD(circuit, target_reg, control_1, control_2, addend, circuit_size, 1)
        self._create_inverse_QFT(circuit, target_reg, circuit_size, 0)
        circuit.x(target_reg[circuit_size - 1])
        circuit.cx(target_reg[circuit_size - 1], aux_reg)
        circuit.x(target_reg[circuit_size - 1])
        self._create_QFT(circuit, target_reg, circuit_size, 0)
        self._ccphiADD(circuit, target_reg, control_1, control_2, addend, circuit_size, 0)

    def _ccphiADDmodN_inv(self, circuit, target_reg, control_1, control_2, aux, base, semiprime, circuit_size):
        """Inverse of doubly controlled modular addition by a"""
        self._ccphiADD(circuit, target_reg, control_1, control_2, base, circuit_size, 1)
        self._create_inverse_QFT(circuit, target_reg, circuit_size, 0)
        circuit.x(target_reg[circuit_size - 1])
        circuit.cx(target_reg[circuit_size - 1], aux)
        circuit.x(target_reg[circuit_size - 1])
        self._create_QFT(circuit, target_reg, circuit_size, 0)
        self._ccphiADD(circuit, target_reg, control_1, control_2, base, circuit_size, 0)
        self._cphiADD(circuit, target_reg, aux, semiprime, circuit_size, 1)
        self._create_inverse_QFT(circuit, target_reg, circuit_size, 0)
        circuit.cx(target_reg[circuit_size - 1], aux)
        self._create_QFT(circuit, target_reg, circuit_size, 0)
        self._phiADD(circuit, target_reg, semiprime, circuit_size, 0)
        self._ccphiADD(circuit, target_reg, control_1, control_2, base, circuit_size, 1)

    def _cMULTmodN(
        self,
        circuit: QuantumCircuit,
        control_qubit,
        classical_reg,
        aux_reg,
        factor,
        semiprime,
        circuit_size,
    ):
        self._create_QFT(circuit, aux_reg, circuit_size + 1, 0)
        """Circuit that implements single controlled modular multiplication by a"""
        for i in range(0, circuit_size):
            self._ccphiADDmodN(
                circuit,
                aux_reg,
                classical_reg[i],
                control_qubit,
                aux_reg[circuit_size + 1],
                (2**i) * factor % semiprime,
                semiprime,
                circuit_size + 1,
            )
        self._create_inverse_QFT(circuit, aux_reg, circuit_size + 1, 0)

        for i in range(0, circuit_size):
            circuit.cswap(control_qubit, classical_reg[i], aux_reg[i])

        a_inv = self._modinv(factor, semiprime)
        self._create_QFT(circuit, aux_reg, circuit_size + 1, 0)
        i = circuit_size - 1
        while i >= 0:
            self._ccphiADDmodN_inv(
                circuit,
                aux_reg,
                classical_reg[i],
                control_qubit,
                aux_reg[circuit_size + 1],
                np.power(2, i) * a_inv % semiprime,
                semiprime,
                circuit_size + 1,
            )
            i -= 1
        self._create_inverse_QFT(circuit, aux_reg, circuit_size + 1, 0)

    def _run_qc(self, base, shots):

        aux = QuantumRegister(self.circuit_size+2)
        up_reg = QuantumRegister(2*self.circuit_size)
        down_reg = QuantumRegister(self.circuit_size)
        up_classic = ClassicalRegister(2*self.circuit_size)
        circuit = QuantumCircuit(down_reg, up_reg, aux, up_classic)

        circuit.h(up_reg)
        circuit.x(down_reg[0])

        for i in range(0, 2*self.circuit_size):
            self._cMULTmodN(circuit, up_reg[i], down_reg, aux, int(pow(base, pow(2, i))), self.semiprime, self.circuit_size)

        self._create_inverse_QFT(circuit, up_reg, 2 * self.circuit_size, 1)
        circuit.measure(up_reg,up_classic)

        qr_backend = QrBackendV2(self.provider, num_qubits = circuit.num_qubits)
        qc_transpiled = transpile(circuit, qr_backend, initial_layout=[i for i in range(0, circuit.num_qubits)], optimization_level=3)

        print(f"Start {base}")
        job = qr_backend.run(qc_transpiled, shots = shots)
        print(f"End {base}")
        result = job.result()
        self.counts[base] = result.get_counts()

    def _bitstring_to_period(self, bitstring):
        phase = int(bitstring, 2) / np.power(2, 2 * self.circuit_size)
        period = Fraction(phase).limit_denominator(self.semiprime).denominator
        return period

    def _classical_factorization(self, base, period):
        if period % 2 == 1:
            return
        
        comp_1 = base ** (period // 2) - 1
        comp_2 = base ** (period // 2) + 1

        if comp_1 % self.semiprime == 0 or comp_2 % self.semiprime == 0:
            return
        
        factor_1 = math.gcd(comp_1, self.semiprime)
        factor_2 = math.gcd(comp_2, self.semiprime)
        if factor_1 in [1, self.semiprime] or factor_2 in [1, self.semiprime]:
            return

        return factor_1, factor_2

    def find_factors(self, base, top_counts = 5, max_attempts = 128):
        self._run_qc(base, max_attempts)
        # check most frequent bitstrings 
        for bitstring in sorted(self.counts[base], key=self.counts[base].get, reverse=True)[:top_counts]:
            period = self._bitstring_to_period(bitstring)
            factors = self._classical_factorization(base, period)
            if factors is not None:
                self.factors.append(factors)

    def factorize(self, max_base_attempts = 5):
        for i, base in enumerate(self.coprimes[:max_base_attempts]):
            self.find_factors(base)

        return self.factors

In [ ]:
factorizer = ShorsFactorizer(899, qr_provider)
factorizer.factorize()